<a href="https://colab.research.google.com/github/janaahmeed/codealpha_tasks/blob/main/Code_Alpha_Task3_(Handwritten_Character_Recognition_).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import joblib
from PIL import Image
import kagglehub

DATA_FILE = "handwritten_features.pkl"

# If already processed → LOAD (FAST)
if os.path.exists(DATA_FILE):
    print("Loading cached features...")
    X, y = joblib.load(DATA_FILE)

else:
    print("Processing images (one-time)...")

    dataset_path = kagglehub.dataset_download(
        "vaibhao/handwritten-characters"
    )

    X, y = [], []

    for root, dirs, files in os.walk(dataset_path):
        label = os.path.basename(root)

        if len(files) == 0:
            continue

        for file in files:
            if file.lower().endswith((".png", ".jpg", ".jpeg")):
                img_path = os.path.join(root, file)

                img = Image.open(img_path).convert("RGB")
                img = img.resize((128, 128))
                img = np.array(img, dtype=np.float32) / 255.0


                X.append(img)
                y.append(label)

    X = np.array(X)
    y = np.array(y)

    joblib.dump((X, y), DATA_FILE)
    print("Features saved.")

print("X shape:", X.shape)
print("y shape:", y.shape)


Processing images (one-time)...


In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder=LabelEncoder()
y_encoded=encoder.fit_transform(y, drop_first=True)
y_onehot= to_categorical(y_encoded)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
sns.countplot(np.unique(y))
plt.show()



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,  # Characters shouldn't flip
    fill_mode='nearest'
)

# Example: Fit the generator to your data
datagen.fit(X)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

num_characters = y_encoded.shape[1]

model = Sequential()

# Block 1
model.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

# Block 2
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

# Block 3
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

# Fully connected layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_characters, activation='softmax'))  # Output layer

# Compile
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.4f}")

y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report( y_test, y_pred, target_names=le.classes_))


In [ ]:

cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Emotion Confusion Matrix")
plt.show()
